In [1]:
import numpy as np
from modified_louvain_method import *
from modified_spectral_method import *
import copy
import random

# Theory
---

1. Time Series and Covariance

    XiXi​ represents the time series for node ii, and Cov[Xi,Xj]Cov[Xi​,Xj​] is the covariance between the time series XiXi​ and XjXj​.
    When defining the renormalized interaction between two communities AA and BB, the approach should be:
        Aggregate the time series within each community.
        Calculate the covariance between these aggregate time series.

This respects the bilinear property of covariance and ensures that the calculation captures the structure of the correlation between the communities.

The earlier formula summing pairwise covariances (∑i∈A∑j∈BCov[Xi,Xj]∑i∈A​∑j∈B​Cov[Xi​,Xj​]) is mathematically equivalent to this approach because of the bilinear property of covariance. However, the "aggregate time series and then calculate covariance" approach provides a cleaner and more intuitive interpretation in the context of time series data.

By working directly with the aggregated time series:

    You treat the community as a single "meta-node" with its own time series.
    This naturally extends the Louvain method to time-series-based networks.


We therefore find that, for a graph composed of financial time series, renormalized interactions have a correct inter- pretation in terms of covariances, rather than correlations. They also show that the summation of a group of time series yields something that resembles an index fund of the set of stocks, so the concept of aggregating nodes maintains a strong grounding in reality.


In [2]:
#Community 1
time_series_1 = np.array([1, 2, 3, 4, 5])
time_series_2 = np.array([5, 1, 3, 10, 1])
community_1 = [0,1]

#Community 2
time_series_3 = np.array([1, 9, 10, 7, 9])
time_series_4 = np.array([1, 2, 3, 2, 8])
community_2 = [2,3]

time_series_matrix = np.stack((time_series_1, time_series_2,time_series_3, time_series_4), axis=0)


cov_matrix = np.cov(time_series_matrix) 
print("Original covariance matrix: ")
print(cov_matrix)

#Calculate covariance between community 1 and 2
cov_comm1_comm2 = 0
for i in community_1:
    for j in community_2:
        cov_comm1_comm2 += cov_matrix[i][j]
    
    
print("Covariance between community 1 and 2: ", cov_comm1_comm2)



Original covariance matrix: 
[[ 2.5   0.25  3.5   3.5 ]
 [ 0.25 14.   -5.25 -5.  ]
 [ 3.5  -5.25 13.2   4.95]
 [ 3.5  -5.    4.95  7.7 ]]
Covariance between community 1 and 2:  -3.25


In [13]:
#Agregate time series community 1
time_series_1_agregate = time_series_1 + time_series_2

#Agregate time series community 2
time_series_2_agregate = time_series_3 + time_series_4


time_series_matrix_agregate = np.stack((time_series_1_agregate, time_series_2_agregate), axis=0)

cov_matrix_agregate = np.cov(time_series_matrix_agregate)
cov_matrix_agregate

array([[17.  , -3.25],
       [-3.25, 30.8 ]])

A note about the final formula:

Efficiency of This Calculation:

    Localized Updates:
        The formula only depends on C~IJ(l)C~IJ(l)​ and C~IJ′(l)C~IJ′(l)​, which are the total interactions between II and the hypernodes in JJ and J′J′, respectively.
        This avoids recomputing the modularity for the entire network.

    Precomputed Interactions:
        In practice, the interactions C~IA(l)C~IA(l)​ between II and any other hypernode AA are typically precomputed, allowing C~IJ(l)C~IJ(l)​ and C~IJ′(l)C~IJ′(l)​ to be calculated quickly by summing over the relevant hypernodes in JJ and J′J′.

# Modified Louvain Method Implementation
---

In [2]:
correlation_matrix,T,N,company_names = create_correlation_matrix('returns_standardized.csv')  
C_g = calculate_C_g(correlation_matrix,T,N) #Modularity matrix
#
#Extract the covariance matrix of the first 4 companies (for testing purposes)
#C_g_slice = C_g[0:4,0:4]

In [3]:
#Check its symmetric!
print(C_g[0,5])
print(C_g[5,0])


-0.0672402125997696
-0.0672402125997696


In [89]:
## Adjusted for randomness

#TODO: Adjust change in modularity formula --- DONE ---
#TODO: Fix issue with map --- DONE ---
#TODO: Adjust logic for empty communities --- DONE ---
#TODO: Optimize algorithm
#TODO: Hypernodes 
# TODO: Check if we need to add i<=j in the modularity calculation 



def calculate_modularity_with_potential_move(node,current_index,neighbour_community_index,communities, modularity_matrix):
    
    "Calculate the modularity of a potential move of a node from its current community to a neighbour community"

    # Get the current and neighbor communities
    current_community = communities[current_index]
    neighbour_community = communities[neighbour_community_index]

    #CALCULATE MODULARITY GAIN FROM ADDING NODE TO NEW COMMUNITY
    #---------------------------------------------------

    # Calculate old modularity of the neighbor community
    # old_modularity_new_community = sum(
    #     modularity_matrix[i][j] 
    #     for i in neighbour_community
    #     for j in neighbour_community
    #     if i <= j
    # )

    # Calculate new modularity after adding the node to the neighbor community
    temp_neighbour_community = neighbour_community + [node] #We create a temp variable to avoid modifying the original community
    new_modularity_new_community = sum(  #The new modularity can be computed by adding the modularity without the node to the additional gain after adding the node
        modularity_matrix[node][j] #We only check pairwise correlations with the node we just added to the community
        #for i in temp_neighbour_community
        for j in temp_neighbour_community
        #if i <= j
    )

    #change_from_addition = new_modularity_new_community - old_modularity_new_community
    change_from_addition  = new_modularity_new_community

    #CALCULATE MODULARITY LOSS FROM REMOVING NODE FROM CURRENT COMMUNITY
    #---------------------------------------------------

    old_modularity_current_community = sum(
        modularity_matrix[i][j]
        for i in current_community
        for j in current_community
        if i <= j
    )

    temp_current_community = copy.deepcopy(current_community) #We create a temp variable to avoid modifying the original community
    temp_current_community.remove([node]) #Remove node from old community

    new_modularity_current_communtity = sum(
        modularity_matrix[i][j]
        for i in temp_current_community
        for j in temp_current_community
        if i <= j
    )


    change_from_removal = old_modularity_current_community - new_modularity_current_communtity



    # Normalize the modularity change
    c_norm = np.sum(modularity_matrix)

    modularity_change = (change_from_addition - change_from_removal) / c_norm

    return modularity_change


def phase_1(communities, modularity_matrix):

    
    nodes = [node for community in communities for node in community]  # Flatten all nodes into a single list
    community_map = {node: index for index, community in enumerate(communities) for node in community}  # Map nodes to their communities
    
    moved = True
    iteration = 0

    while moved == True:  # Continue until no moves improve modularities

        # print(f"ITERATION {iteration}")
        # print("-----------------")
        
        iteration += 1

        moved = False
        random.shuffle(nodes)  # Randomly shuffle the nodes
        
        for node in nodes:

            #print("COMMUNITY MAP BELOW:")
            #print(community_map)

            current_index = community_map[node]  # Find the node's current community index
            #print("Current index",current_index)
            current_community = communities[current_index]
            
            #print("Node:", node)
            #print("Current Node Communty:", current_community)
            
            max_modularity = 0
            best_community = None
            
            for j, neighbor_community in enumerate(communities):

                if not neighbor_community or neighbor_community == current_community:
                    continue  # Skip empty or identical communities
                
                modularity_change = calculate_modularity_with_potential_move(node, current_index, j, communities, modularity_matrix)
                
                if modularity_change > max_modularity:
                    max_modularity = modularity_change
                    best_community = neighbor_community
            
            # If a modularity improvement is found, move the node
            if max_modularity > 0 and best_community is not None:
                current_community.remove(node)
                best_community.append(node)
                community_map[node] = communities.index(best_community)  # Update the community map
                moved = True  # Indicate that a move occurred
                #print("MOVED NODE", node, "TO COMMUNITY", communities.index(best_community))

                #print("UPDATED COMMUNITY MAP:")
                #print(community_map)

        # Remove empty communities
        communities = [community for community in communities if community]

        #Update community map with new communities
        community_map = {node: index for index, community in enumerate(communities) for node in community}  # Map nodes to their communities
        
        # print(f"End of iteration {iteration}, Communities: {communities},")
    
    return communities

def phase_2(communities,modularity_matrix):

    "Node aggregation phase"

    # Aggregate nodes into hypernodes

    #df_standardized_returns = pd.read_csv("returns_standardized.csv")

    renormalized_modularity_matrix = np.zeros((len(communities),len(communities))) #If there are 5 communities, this will be a 5x5 matrix

    for community_index, community in enumerate(communities):
        
        hypernode_correlations = [] #Stores the correlation between the current community and all other communities (Including self loops)

        for  neighbour_community in communities:
            
            # if community == neighbour_community:
            #     continue
            
            # Calculate the covariance between the current community and the neighbour community
            cov_comm1_comm2 = 0

            for i in community:
                for j in neighbour_community:
                    cov_comm1_comm2 += modularity_matrix[i][j]
            
            hypernode_correlations.append(cov_comm1_comm2)

        renormalized_modularity_matrix[community_index] = hypernode_correlations
        

    return renormalized_modularity_matrix

def flatten_final_communities(final_communities):

    flattened_communities = []
    for item in final_communities:
        if isinstance(item, list) and all(isinstance(subitem, list) for subitem in item):
            # Flatten one level of nesting
            flattened_item = [subitem for nested in item for subitem in nested]
            flattened_communities.append(flattened_item)
        else:
            flattened_communities.append(item)  # Keep as is
            
    return flattened_communities

def map_hypernodes_to_nodes(hypernode_communities, node_communities):

    final_communities = []

    for community in hypernode_communities:

        final_community = []

        for hyper_node in community:

            node_community = node_communities[hyper_node]
            final_community.append(node_community)
            

        final_communities.append(final_community)

    #Flatten final communities so they are in the same shape as the original communities
    final_communities = flatten_final_communities(final_communities)
    
    return final_communities


def modified_louvain_1(modularity_matrix):

    #Get node indices from matrix
    node_indices = np.arange(modularity_matrix.shape[0])

    #Create initial communities
    initial_pahse1_communities = [[node] for node in node_indices]

    #Phase 1 for nodes
    #--------------------------------

    #Calculate initial communities
    phase1_communities = phase_1(initial_pahse1_communities, modularity_matrix)

    #Phase 2 (aggregation) for hypernodes
    #--------------------------------

    renormalized_modularity_matrix = phase_2(phase1_communities,modularity_matrix)

    hyper_node_indices = np.arange(renormalized_modularity_matrix.shape[0])

    initial_hypernode_communities = [[node] for node in hyper_node_indices]

    #print("Initial hypernode communities", initial_hypernode_communities)

    #Phase 1 for hypernodes
    #--------------------------------

    phase1_hypernode_communities = phase_1(initial_hypernode_communities, renormalized_modularity_matrix)

    #Map back hypernode communities to node communities
    #--------------------------------

    print("Detected", len(phase1_communities), "Initially")

    final_communities = map_hypernodes_to_nodes(phase1_hypernode_communities, phase1_communities)

    print("Final number of communities:", len(final_communities))
    
    return final_communities


In [93]:
louvain_communities = modified_louvain_1(C_g)

Detected 5 Initially
Final number of communities: 4


In [94]:
for community in louvain_communities:
    print(community)

[3, 65, 2, 29]
[21, 77, 33, 52, 43, 1, 53, 75, 11, 39, 64, 25, 35, 49, 41, 30, 34, 4, 20, 38, 32, 44, 28, 87, 59, 23, 84, 58, 85, 56, 10, 12, 88, 36, 80, 51, 63, 45, 24, 42]
[31, 74, 81, 5, 62, 72, 8, 67, 89, 82, 61, 16, 17, 26, 19, 83, 18, 54, 22, 40, 13, 6, 86, 66, 14, 57, 68, 78, 73, 76, 27]
[48, 55, 69, 7, 9, 46, 47, 79, 15, 71, 37, 0, 60, 50, 70]


In [95]:
#Spectral method
spectral_communities, company_communities_spectral, modularities = recursive_spectral_method(C_g, company_names,min_size=2, modularity_threshold=0.00001)

for community in spectral_communities:
    print(community)

[1, 5, 6, 8, 13, 14, 16, 17, 18, 19, 22, 26, 27, 31, 33, 34, 40, 54, 57, 61, 62, 66, 67, 68, 69, 72, 73, 74, 76, 78, 81, 82, 83, 86, 89]
[2, 3, 29, 65]
[0, 7, 9, 37, 47, 48, 55, 60, 70, 79]
[4, 10, 11, 12, 15, 20, 21, 23, 24, 25, 28, 30, 32, 35, 36, 38, 39, 41, 42, 43, 44, 45, 46, 49, 50, 51, 52, 53, 56, 58, 59, 63, 64, 71, 75, 77, 80, 84, 85, 87, 88]


In [96]:
def calculate_global_modularity(communities, modularity_matrix):

    #Calculate the global modularity of the communities

    modularity = 0
    c_norm = np.sum(modularity_matrix)

    for community in communities:
        for i in community:
            for j in community:  
                if i<=j:  #Ensure that we only add each pair of nodes once! 
                    modularity += modularity_matrix[i][j]

    modularity = modularity/c_norm

    return modularity

print("Global modularity of modified Louvain method: ", calculate_global_modularity(louvain_communities, C_g))
print("Global modularity of spectral method: ", calculate_global_modularity(spectral_communities, C_g))

Global modularity of modified Louvain method:  15.185476426292789
Global modularity of spectral method:  14.87421549519166


In [97]:
company_communities_louvain = []

for partition in louvain_communities:
    company_list = []
    for i in partition:
        company_list.append(company_names[i])
    company_communities_louvain.append(company_list)


In [98]:
for company_community in company_communities_louvain:
    print(company_community)

['ANTO.L', 'RIO.L', 'AAL.L', 'FRES.L']
['DCC.L', 'SPX.L', 'HL.L', 'MRO.L', 'ITRK.L', 'ADM.L', 'MNDI.L', 'SMDS.L', 'BEZ.L', 'IMI.L', 'RMV.L', 'ENT.L', 'HSX.L', 'LMP.L', 'INF.L', 'GLEN.L', 'HIK.L', 'AHT.L', 'CRDA.L', 'IHG.L', 'HLMA.L', 'JD.L', 'FRAS.L', 'WEIR.L', 'PHNX.L', 'DPLM.L', 'UTG.L', 'PSN.L', 'VTY.L', 'NXT.L', 'BDEV.L', 'BKG.L', 'WTB.L', 'HWDN.L', 'TW.L', 'MKS.L', 'RTO.L', 'KGF.L', 'EZJ.L', 'IAG.L']
['GSK.L', 'SN.L', 'TSCO.L', 'ABF.L', 'REL.L', 'SVT.L', 'BA.L', 'SGE.L', 'WPP.L', 'ULVR.L', 'RKT.L', 'BT-A.L', 'BNZL.L', 'EXPN.L', 'CPG.L', 'UU.L', 'CNA.L', 'NG.L', 'DGE.L', 'IMB.L', 'BP.L', 'AZN.L', 'VOD.L', 'RR.L', 'BATS.L', 'PSON.L', 'SBRY.L', 'SSE.L', 'SHEL.L', 'SMIN.L', 'FCIT.L']
['LLOY.L', 'NWG.L', 'SDR.L', 'AV.L', 'BARC.L', 'LAND.L', 'LGEN.L', 'STAN.L', 'BLND.L', 'SGRO.L', 'HSBA.L', 'III.L', 'PRU.L', 'LSEG.L', 'SMT.L']


In [74]:
for company_community in company_communities_spectral:
    print(company_community)

['ADM.L', 'ABF.L', 'AZN.L', 'BA.L', 'BP.L', 'BATS.L', 'BT-A.L', 'BNZL.L', 'CNA.L', 'CPG.L', 'DGE.L', 'EXPN.L', 'FCIT.L', 'GSK.L', 'HL.L', 'HIK.L', 'IMB.L', 'NG.L', 'PSON.L', 'RKT.L', 'REL.L', 'RR.L', 'SGE.L', 'SBRY.L', 'SDR.L', 'SVT.L', 'SHEL.L', 'SN.L', 'SMIN.L', 'SSE.L', 'TSCO.L', 'ULVR.L', 'UU.L', 'VOD.L', 'WPP.L']
['AAL.L', 'ANTO.L', 'FRES.L', 'RIO.L']
['III.L', 'AV.L', 'BARC.L', 'HSBA.L', 'LGEN.L', 'LLOY.L', 'NWG.L', 'PRU.L', 'SMT.L', 'STAN.L']
['AHT.L', 'BDEV.L', 'BEZ.L', 'BKG.L', 'BLND.L', 'CRDA.L', 'DCC.L', 'DPLM.L', 'EZJ.L', 'ENT.L', 'FRAS.L', 'GLEN.L', 'HLMA.L', 'HSX.L', 'HWDN.L', 'IHG.L', 'IMI.L', 'INF.L', 'IAG.L', 'ITRK.L', 'JD.L', 'KGF.L', 'LAND.L', 'LMP.L', 'LSEG.L', 'MKS.L', 'MRO.L', 'MNDI.L', 'NXT.L', 'PSN.L', 'PHNX.L', 'RTO.L', 'RMV.L', 'SGRO.L', 'SMDS.L', 'SPX.L', 'TW.L', 'UTG.L', 'VTY.L', 'WEIR.L', 'WTB.L']


## NOTE: LISTS ARE MUTABLE OBJECTS!!!

In [ ]:
test_list = [1,2,3,4]

def test_function(test_list):
    test_list.append(5)
    
test_function(test_list)

print(test_list)



[1, 2, 3, 4]


In [ ]:
original_list = [[1],[2],[3],[4]]

deep_copy_list = copy.deepcopy(original_list) # Deep copy
shallow_copy_list = original_list.copy() # Shallow copy


#Testing
#--------------------------
deep_copy_list[0].append(5)

print(original_list)
print(deep_copy_list)

[[1], [2], [3], [4]]
[[1, 5], [2], [3], [4]]


In [ ]:
original_list = [1, 2, 3]

for item in original_list[:]:
    print(item)
    if item == 2:
        original_list.remove(item)
print(original_list)

1
2
[1, 3]


# Shallow copies
The use of [:] to create a copy of the list ensures that the loops iterate over the original elements of list_test even if the list is modified during the iteration. This prevents issues that can arise from modifying a list while iterating over it directly.

In [56]:
list_test = [1,2,3,4]

for i in list_test[:]:
    for j in list_test[:]:
        print(j)
        if j == 1:
            list_test.remove(j)
            print(j, "removed")
        

1
1 removed
2
3
4
2
3
4
2
3
4
2
3
4


In [31]:
modified_louvain(C_g_slice)



0.10804988581676901
0.21548050242159764
0.14085445928586848
0.14149913944138
[0]
[1]
[2]
[3]


In [31]:
# Reverse list
#--------------------------
list_test = [1,2,3,4]

for i in reversed(list_test):
    if i == 2:
        list_test.remove(i)
    else:
        print(i)
        

4
3
1
